In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional, Input
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Concatenate, GlobalAveragePooling1D
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import resample, class_weight
import os
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# ── CONFIG ─────────────────────────────
SEQUENCE_LENGTH = 60
LANDMARKS = 63  # Total landmarks: 21 hand landmarks × 3 coordinates (x, y, z)
GESTURE_LABELS = ["swipe_left", "swipe_right", "swipe_up", "swipe_down", "screenshot", "drop", "like", "dislike", "sos", "peace"]
DATASET_DIR = "dataset"
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
AUGMENTATION_NOISE = 0.01
NUM_CLASSES = len(GESTURE_LABELS)

# Define static and dynamic gestures
STATIC_GESTURES = ["like", "dislike", "peace"]
DYNAMIC_GESTURES = [g for g in GESTURE_LABELS if g not in STATIC_GESTURES]

# ── LOAD DATA FROM CSV FILES ───────────
print("Loading data...")
X_data = []
y_labels = []

# Modified loading code to handle the structure created by the recording script
for label_idx, gesture in enumerate(GESTURE_LABELS):
    # Check both root directory and type-specific subdirectories
    possible_dirs = [
        os.path.join(DATASET_DIR, gesture),  # Direct gesture folder
        os.path.join(DATASET_DIR, "static", gesture),  # Static gestures
        os.path.join(DATASET_DIR, "dynamic", gesture)  # Dynamic gestures
    ]
    
    for gesture_dir in possible_dirs:
        if not os.path.exists(gesture_dir):
            continue
            
        gesture_files = [f for f in os.listdir(gesture_dir) if f.endswith('.csv')]
        print(f"Found {len(gesture_files)} sequences for gesture '{gesture}' in {gesture_dir}")
        
        for file_name in gesture_files:
            file_path = os.path.join(gesture_dir, file_name)
            try:
                sequence = pd.read_csv(file_path, header=None).values
                if sequence.shape != (SEQUENCE_LENGTH, LANDMARKS):
                    print(f"⚠️ Skipping {file_path}: Expected shape {(SEQUENCE_LENGTH, LANDMARKS)}, got {sequence.shape}")
                    continue
                X_data.append(sequence)
                y_labels.append(label_idx)
            except Exception as e:
                print(f"⚠️ Error loading {file_path}: {e}")
                continue

# Convert to NumPy arrays
if not X_data:
    print("❌ No valid data found. Please check your dataset directories.")
    exit()

X_data = np.array(X_data, dtype=np.float32)
y_labels = np.array(y_labels)

# Update GESTURE_LABELS based on loaded data
loaded_labels = np.unique(y_labels)
if len(loaded_labels) != len(GESTURE_LABELS):
    print(f"⚠️ Mismatch: Expected {len(GESTURE_LABELS)} gestures, but only {len(loaded_labels)} were loaded.")
    missing_labels = set(range(len(GESTURE_LABELS))) - set(loaded_labels)
    missing_gestures = [GESTURE_LABELS[idx] for idx in missing_labels]
    print(f"Missing gestures: {missing_gestures}")
    GESTURE_LABELS = [GESTURE_LABELS[idx] for idx in sorted(loaded_labels)]
    STATIC_GESTURES = [g for g in STATIC_GESTURES if g in GESTURE_LABELS]
    DYNAMIC_GESTURES = [g for g in DYNAMIC_GESTURES if g in GESTURE_LABELS]
    NUM_CLASSES = len(GESTURE_LABELS)
    print(f"Updated GESTURE_LABELS: {GESTURE_LABELS}")

# Print class distribution before preprocessing
unique, counts = np.unique(y_labels, return_counts=True)
print("\n--- Initial Class Distribution ---")
for i, (label_idx, count) in enumerate(zip(unique, counts)):
    print(f"{GESTURE_LABELS[label_idx]}: {count} samples")
print("-------------------------\n")

# ── DATA PREPROCESSING ─────────────────
print("Preprocessing data...")

# Normalize sequences by subtracting the first frame
X_data_norm = np.array([seq - seq[0] for seq in X_data])

# Extract velocity features (temporal derivatives)
X_velocity = np.zeros_like(X_data_norm)
for i in range(len(X_data_norm)):
    X_velocity[i, 1:, :] = X_data_norm[i, 1:, :] - X_data_norm[i, :-1, :]
    X_velocity[i, 0, :] = 0  # First frame has zero velocity

# Compute magnitude of movement for each landmark
X_magnitude = np.zeros((X_data_norm.shape[0], X_data_norm.shape[1], X_data_norm.shape[2]//3))
for i in range(len(X_data_norm)):
    for j in range(0, X_data_norm.shape[2], 3):
        if j+2 < X_data_norm.shape[2]:  # Ensure we have x, y, z
            x = X_data_norm[i, :, j]
            y = X_data_norm[i, :, j+1]
            z = X_data_norm[i, :, j+2] if (j+2) < X_data_norm.shape[2] else np.zeros_like(x)
            X_magnitude[i, :, j//3] = np.sqrt(x**2 + y**2 + z**2)

# For static gestures, compute the mean pose over the sequence
X_data_static = np.zeros_like(X_data_norm)
for i, label in enumerate(y_labels):
    gesture = GESTURE_LABELS[label]
    if gesture in STATIC_GESTURES:
        # For static gestures, take the average of frames 20-40 (middle of sequence)
        middle_frames = X_data_norm[i, 20:40, :]
        mean_pose = np.mean(middle_frames, axis=0, keepdims=True)
        X_data_static[i] = np.repeat(mean_pose, SEQUENCE_LENGTH, axis=0)
    else:
        X_data_static[i] = X_data_norm[i]

# ── ENHANCED AUGMENTATION ───────────────
def augment_data(X, y, noise_level=AUGMENTATION_NOISE, time_warp_factor=0.2):
    # Add noise
    X_augmented = X.copy()
    noise = np.random.uniform(-noise_level, noise_level, X.shape)
    X_augmented += noise
    
    # Time warping (speed variations)
    X_time_warped = []
    y_time_warped = []
    
    for i, sequence in enumerate(X):
        # Only apply time warping to dynamic gestures
        if GESTURE_LABELS[y[i]] in DYNAMIC_GESTURES:
            # Slow down
            slow_factor = 1 - np.random.uniform(0, time_warp_factor)
            slow_length = min(int(SEQUENCE_LENGTH * slow_factor), SEQUENCE_LENGTH-1)
            indices = np.linspace(0, slow_length-1, SEQUENCE_LENGTH).astype(int)
            slow_sequence = sequence[indices]
            X_time_warped.append(slow_sequence)
            y_time_warped.append(y[i])
            
            # Speed up
            fast_factor = 1 + np.random.uniform(0, time_warp_factor)
            fast_length = min(int(SEQUENCE_LENGTH * fast_factor), SEQUENCE_LENGTH)
            if fast_length > SEQUENCE_LENGTH:
                indices = np.linspace(0, SEQUENCE_LENGTH-1, fast_length).astype(int)
                fast_sequence = sequence[indices[:SEQUENCE_LENGTH]]
            else:
                indices = np.linspace(0, fast_length-1, SEQUENCE_LENGTH).astype(int)
                fast_sequence = np.zeros_like(sequence)
                fast_sequence[:fast_length] = sequence[indices]
                fast_sequence[fast_length:] = sequence[-1]  # Pad with last frame
            X_time_warped.append(fast_sequence)
            y_time_warped.append(y[i])
    
    if X_time_warped:
        X_time_warped = np.array(X_time_warped)
        y_time_warped = np.array(y_time_warped)
        X_augmented = np.vstack([X_augmented, X_time_warped])
        y = np.hstack([y, y_time_warped])
    
    return X_augmented, y

# ── BALANCE DATASET ────────────────────
print("\n--- Balancing dataset ---")
unique, counts = np.unique(y_labels, return_counts=True)
max_samples = max(counts)
X_balanced = []
y_balanced = []

for label_idx in unique:
    X_class = X_data_norm[y_labels == label_idx]
    y_class = y_labels[y_labels == label_idx]
    if len(X_class) < max_samples:
        X_class_resampled, y_class_resampled = resample(
            X_class, y_class,
            replace=True,
            n_samples=max_samples,
            random_state=RANDOM_SEED
        )
        X_balanced.append(X_class_resampled)
        y_balanced.append(y_class_resampled)
    else:
        X_balanced.append(X_class)
        y_balanced.append(y_class)

X_data_norm = np.vstack(X_balanced)
y_labels_balanced = np.hstack(y_balanced)

# Apply augmentation to balanced dataset
X_data_norm, y_labels_balanced = augment_data(X_data_norm, y_labels_balanced)

# Print new class distribution
unique, counts = np.unique(y_labels_balanced, return_counts=True)
print("\n--- Class Distribution After Balancing and Augmentation ---")
for i, (label_idx, count) in enumerate(zip(unique, counts)):
    print(f"{GESTURE_LABELS[label_idx]}: {count} samples")
print("-------------------------\n")

# Convert labels to one-hot encoding
y_data = tf.keras.utils.to_categorical(y_labels_balanced, NUM_CLASSES)

print(f"✅ Loaded and processed {len(X_data_norm)} sequences with shape {X_data_norm.shape}")

# ── ADVANCED MODEL ARCHITECTURE ─────────
print("Building enhanced gesture recognition model...")

def create_attention_layer(query, value, key=None):
    if key is None:
        key = value
    # Multihead attention layer
    return tf.keras.layers.MultiHeadAttention(
        num_heads=4, key_dim=32
    )(query=query, value=value, key=key)

def create_enhanced_model():
    # Main sequence input
    input_seq = Input(shape=(SEQUENCE_LENGTH, LANDMARKS), name='sequence_input')
    
    # === SPATIAL FEATURE EXTRACTION ===
    # CNN Block 1: Capture spatial patterns with different kernel sizes
    conv1 = Conv1D(64, kernel_size=3, padding='same', activation='relu')(input_seq)
    conv1 = BatchNormalization()(conv1)
    conv1 = SpatialDropout1D(0.3)(conv1)
    
    conv2 = Conv1D(64, kernel_size=5, padding='same', activation='relu')(input_seq)
    conv2 = BatchNormalization()(conv2)
    conv2 = SpatialDropout1D(0.3)(conv2)
    
    conv3 = Conv1D(64, kernel_size=7, padding='same', activation='relu')(input_seq)
    conv3 = BatchNormalization()(conv3)
    conv3 = SpatialDropout1D(0.3)(conv3)
    
    # Combine CNN features
    cnn_features = Concatenate()([conv1, conv2, conv3])
    cnn_features = MaxPooling1D(pool_size=2)(cnn_features)
    
    # === TEMPORAL FEATURE EXTRACTION ===
    # LSTM Block: Capture temporal relationships
    lstm = Bidirectional(LSTM(128, return_sequences=True))(cnn_features)
    lstm = LayerNormalization()(lstm)
    lstm = SpatialDropout1D(0.4)(lstm)
    
    lstm = Bidirectional(LSTM(64, return_sequences=True))(lstm)
    lstm = LayerNormalization()(lstm)
    lstm = SpatialDropout1D(0.3)(lstm)
    
    # Self-attention mechanism to focus on important parts of the sequence
    attention_output = create_attention_layer(lstm, lstm)
    attention_output = LayerNormalization()(attention_output)
    
    # Global feature extraction
    global_max = GlobalAveragePooling1D()(attention_output)
    
    # === FINAL CLASSIFICATION LAYERS ===
    dense = Dense(128, activation='relu')(global_max)
    dense = BatchNormalization()(dense)
    dense = Dropout(0.5)(dense)
    
    dense = Dense(64, activation='relu')(dense)
    dense = BatchNormalization()(dense)
    dense = Dropout(0.4)(dense)
    
    output = Dense(NUM_CLASSES, activation='softmax')(dense)
    
    model = Model(inputs=input_seq, outputs=output)
    
    optimizer = Adam(learning_rate=0.0005)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ── K-FOLD CROSS VALIDATION ───────────
def perform_kfold_training(X, y, n_splits=5):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    fold_results = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X)):
        print(f"\n--- Training Fold {fold+1}/{n_splits} ---")
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]
        
        # Compute class weights for this fold
        y_train_classes = np.argmax(y_train_fold, axis=1)
        class_weights = class_weight.compute_class_weight(
            class_weight='balanced',
            classes=np.unique(y_train_classes),
            y=y_train_classes
        )
        class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
        
        # Create and train model for this fold
        model = create_enhanced_model()
        if fold == 0:  # Only print summary for first fold
            model.summary()
        
        early_stopping = EarlyStopping(
            monitor='val_accuracy',
            patience=30,
            restore_best_weights=True,
            verbose=1
        )
        
        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=15,
            min_lr=0.0000001,
            verbose=1
        )
        
        model_checkpoint = ModelCheckpoint(
            filepath=f'best_model_fold{fold+1}.h5',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
        
        history = model.fit(
            X_train_fold, y_train_fold,
            validation_data=(X_val_fold, y_val_fold),
            epochs=150,
            batch_size=32,
            callbacks=[early_stopping, reduce_lr, model_checkpoint],
            class_weight=class_weights_dict,
            verbose=1
        )
        
        # Evaluate fold performance
        y_val_pred = model.predict(X_val_fold, verbose=0)
        y_val_pred_classes = np.argmax(y_val_pred, axis=1)
        y_val_true_classes = np.argmax(y_val_fold, axis=1)
        
        print(f"\nFold {fold+1} Classification Report:")
        report = classification_report(
            y_val_true_classes, 
            y_val_pred_classes, 
            target_names=GESTURE_LABELS, 
            output_dict=True
        )
        fold_results.append({
            'model': model,
            'history': history,
            'report': report,
            'accuracy': report['accuracy']
        })
        
        print(f"Fold {fold+1} Accuracy: {report['accuracy']:.4f}")
        
        # Plot confusion matrix for this fold
        plt.figure(figsize=(10, 8))
        cm = confusion_matrix(y_val_true_classes, y_val_pred_classes)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                   xticklabels=GESTURE_LABELS, yticklabels=GESTURE_LABELS)
        plt.title(f'Confusion Matrix - Fold {fold+1}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f'confusion_matrix_fold{fold+1}.png')
        plt.close()
    
    return fold_results

# ── TRAIN WITH CROSS-VALIDATION ─────────────────
print("\n🧠 Starting k-fold cross-validation training...")
# Create train/validation split for final evaluation
X_train, X_val, y_train, y_val = train_test_split(
    X_data_norm, y_data,
    test_size=0.2,
    random_state=RANDOM_SEED,
    stratify=np.argmax(y_data, axis=1)
)

# Run a quick sanity check
print(f"\nSanity check - Training split shape: {X_train.shape}")
print(f"Sanity check - Validation split shape: {X_val.shape}")

if NUM_CLASSES == 0 or X_data_norm.shape[0] == 0:
    print("❌ Error: No valid data for training. Check your dataset directories.")
    exit()

# First run k-fold cross-validation
fold_results = perform_kfold_training(X_data_norm, y_data, n_splits=5)

# Find best performing fold model
best_fold = max(range(len(fold_results)), key=lambda i: fold_results[i]['accuracy'])
best_model = fold_results[best_fold]['model']
print(f"\n🏆 Best model from fold {best_fold+1} with accuracy: {fold_results[best_fold]['accuracy']:.4f}")

# ── TRAIN FINAL MODEL ON ENTIRE DATASET ─
print("\n🔄 Training final model on entire dataset...")
final_model = create_enhanced_model()

# Use parameters from the best fold model
best_model.save_weights('best_fold_weights.weights.h5')
final_model.load_weights('best_fold_weights.weights.h5')

# Further train on entire dataset
y_classes = np.argmax(y_data, axis=1)
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_classes),
    y=y_classes
)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Callbacks for final model
early_stopping = EarlyStopping(
    monitor='loss',
    patience=30,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='loss',
    factor=0.5,
    patience=15,
    min_lr=0.0000001,
    verbose=1
)

model_checkpoint = ModelCheckpoint(
    filepath='best_model.h5',
    monitor='loss',
    save_best_only=True,
    verbose=1
)

final_history = final_model.fit(
    X_data_norm, y_data,
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr, model_checkpoint],
    class_weight=class_weights_dict,
    verbose=1
)

# Evaluate on the entire dataset
loss, accuracy = final_model.evaluate(X_data_norm, y_data, verbose=1)
print(f"\nFinal model accuracy on entire dataset: {accuracy*100:.2f}%")

# ── SAVE LABELS ────────────────────────
np.save("labels.npy", np.array(GESTURE_LABELS))
print("✅ Saved labels.npy")

# ── CONVERT TO TFLITE ──────────────────
print("🔄 Converting to TFLite...")
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()

# ── SAVE TFLITE MODEL ──────────────────
with open("gesture_model.tflite", "wb") as f:
    f.write(tflite_model)
print("✅ Saved gesture_model.tflite")

# ── PLOT TRAINING HISTORY ──────────────
plt.figure(figsize=(12, 10))

# Plot accuracy and loss for each fold
plt.subplot(2, 1, 1)
for i, result in enumerate(fold_results):
    plt.plot(result['history'].history['accuracy'], label=f'Fold {i+1} Train')
    plt.plot(result['history'].history['val_accuracy'], label=f'Fold {i+1} Val', linestyle='--')
plt.title('K-Fold Cross-Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(2, 1, 2)
for i, result in enumerate(fold_results):
    plt.plot(result['history'].history['loss'], label=f'Fold {i+1} Train')
    plt.plot(result['history'].history['val_loss'], label=f'Fold {i+1} Val', linestyle='--')
plt.title('K-Fold Cross-Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('kfold_training_history.png')
plt.close()

print("🎉 Training complete. Use `gesture_model.tflite` and `labels.npy` for inference.")

Loading data...
Found 50 sequences for gesture 'swipe_left' in dataset\dynamic\swipe_left
Found 50 sequences for gesture 'swipe_right' in dataset\dynamic\swipe_right
Found 50 sequences for gesture 'swipe_up' in dataset\dynamic\swipe_up
Found 50 sequences for gesture 'swipe_down' in dataset\dynamic\swipe_down
Found 50 sequences for gesture 'screenshot' in dataset\dynamic\screenshot
Found 50 sequences for gesture 'drop' in dataset\dynamic\drop
Found 50 sequences for gesture 'like' in dataset\static\like
Found 50 sequences for gesture 'dislike' in dataset\static\dislike
Found 50 sequences for gesture 'sos' in dataset\dynamic\sos
Found 50 sequences for gesture 'peace' in dataset\static\peace

--- Initial Class Distribution ---
swipe_left: 50 samples
swipe_right: 50 samples
swipe_up: 50 samples
swipe_down: 50 samples
screenshot: 50 samples
drop: 50 samples
like: 50 samples
dislike: 50 samples
sos: 50 samples
peace: 50 samples
-------------------------

Preprocessing data...

--- Balancing d

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ sequence_input (InputLayer)   │ (None, 60, 63)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d (Conv1D)               │ (None, 60, 64)            │          12,160 │ sequence_input[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_1 (Conv1D)             │ (None, 60, 64)            │          20,224 │ sequence_input[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_2 (Conv1D)             │ (None, 60, 64)            │          28,288 │ sequence_input[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 60, 64)            │             256 │ conv1d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 60, 64)            │             256 │ conv1d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 60, 64)            │             256 │ conv1d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ spatial_dropout1d             │ (None, 60, 64)            │               0 │ batch_normalization[0][0]  │
│ (SpatialDropout1D)            │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ spatial_dropout1d_1           │ (None, 60, 64)            │               0 │ batch_normalization_1[0][… │
│ (SpatialDropout1D)            │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ spatial_dropout1d_2           │ (None, 60, 64)            │               0 │ batch_normalization_2[0][… │
│ (SpatialDropout1D)            │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 60, 192)           │               0 │ spatial_dropout1d[0][0],   │
│                               │                           │                 │ spatial_dropout1d_1[0][0], │
│                               │                           │                 │ spatial_dropout1d_2[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling1d (MaxPooling1D)  │ (None, 30, 192)           │               0 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional (Bidirectional) │ (None, 30, 256)           │         328,704 │ max_pooling1d[0][0]        │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 647,754 (2.47 MB)

 Trainable params: 646,986 (2.47 MB)

...

38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 1.0000 - loss: 0.0030 - learning_rate: 2.5000e-04
Epoch 86/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9998 - loss: 0.0031
Epoch 86: loss improved from 0.00345 to 0.00311, saving model to best_model.h5


38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.9997 - loss: 0.0031 - learning_rate: 2.5000e-04
Epoch 87/100
37/38 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9988 - loss: 0.0070
Epoch 87: loss did not improve from 0.00311
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.9988 - loss: 0.0070 - learning_rate: 2.5000e-04
Epoch 88/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9976 - loss: 0.0571
Epoch 88: loss did not improve from 0.00311
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9975 - loss: 0.0569 - learning_rate: 2.5000e-04
Epoch 89/100
36/38 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9993 - loss: 0.0141
Epoch 89: loss did not improve from 0.00311
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9991 - loss: 0.0151 - learning_rate: 2.5000e-04
Epoch 90/100
37/38 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9999 - loss: 0.0044
Epoch 90: loss did not improve from 0.00311
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.9998 - loss: 0

38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 1.0000 - loss: 0.0029 - learning_rate: 2.5000e-04
Epoch 97/100
37/38 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9977 - loss: 0.0053
Epoch 97: loss did not improve from 0.00293
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9977 - loss: 0.0053 - learning_rate: 2.5000e-04
Epoch 98/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9991 - loss: 0.0210
Epoch 98: loss did not improve from 0.00293
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.9991 - loss: 0.0213 - learning_rate: 2.5000e-04
Epoch 99/100
37/38 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 0.0027
Epoch 99: loss did not improve from 0.00293
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 1.0000 - loss: 0.0028 - learning_rate: 2.5000e-04
Epoch 100/100
37/38 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9994 - loss: 0.0065
Epoch 100: loss did not improve from 0.00293
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.9994 - loss:

INFO:tensorflow:Assets written to: C:\Users\elroy\AppData\Local\Temp\tmpo4il44uq\assets


Saved artifact at 'C:\Users\elroy\AppData\Local\Temp\tmpo4il44uq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 60, 63), dtype=tf.float32, name='sequence_input')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  2860628421168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2860628418000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2859140512608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2859140513312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2859140506096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2859140502928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2860628421696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2860628423104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2860628416064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2860628421344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2860628